In [5]:
import numpy as np 
import io
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset
from PIL import Image
import torch
import torch.nn as nn
from matplotlib import gridspec


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

# Load the "emoji-dataset"
dataset = load_dataset("valhalla/emoji-dataset")

# Split the dataset into training, testing, and validation sets
train_ratio = 0.6
test_ratio = 0.2
validation_ratio = 0.2

# Use the train_test_split function to create the splits
splits = dataset["train"].train_test_split(
    test_size=test_ratio,
    train_size=train_ratio,
    shuffle=True,
    # split_seed=42  # You can change this seed for reproducibility
)

# Access the resulting splits
train_dataset = splits["train"]
test_dataset = splits["test"]

# Further split the train_dataset for validation
train_splits = train_dataset.train_test_split(
    test_size=validation_ratio,
    train_size=1 - validation_ratio,
    shuffle=True,
    # split_seed=42
)

# Access the resulting splits
train_dataset = train_splits["train"]
validation_dataset = train_splits["test"]

# Print the number of examples in each split
print(f"Train: {len(train_dataset)} examples")
print(f"Test: {len(test_dataset)} examples")
print(f"Validation: {len(validation_dataset)} examples")


Train: 1319 examples
Test: 550 examples
Validation: 330 examples


In [16]:
first_example = train_dataset[0]
# first_example = Image.open(io.BytesIO(train_dataset.iloc[0]['image']['bytes']))

first_example
# Access the image from the dictionary
image = first_example['image']

# Show the image
image.size
plt.imshow(image)


(256, 256)

In [8]:
def show_images(images):
    images = np.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
    sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))

    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return

Constructing the Generating Neural Network

- fully connected layer that takes an input from noise_dim and outputs to a size of 1024.
- A ReLU activation function.
- Another fully connected layer, this time maintaining a size of 1024.
- nother ReLU activation function.
- A final fully connected layer with a size of 784.
- A TanH activation function, which is used to ensure the output image values are within the range of -1 to 1.

From the Paper:
Architecture guidelines for stable Deep Convolutional GANs
- Replace any pooling layers with strided convolutions (discriminator) and fractional-strided
convolutions (generator).
- Use batchnorm in both the generator and the discriminator.
- Remove fully connected hidden layers for deeper architectures.
- Use ReLU activation in generator for all layers except for the output, which uses Tanh.
- Use LeakyReLU activation in the discriminator for all layers|
- Item 1



In [15]:
def generator_model(input_size, hidden_size, output_size):
    layer1 = [
        nn.Linear(in_features=input_size, out_features=hidden_size, bias=False),
        nn.BatchNorm1d(hidden_size),
        # allows the gradients to flow better through the model 
        nn.LeakyReLU(inplace=True)
    ]
    layer2 = [
        nn.ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(inplace=True)
            
    ]
    layer3 = [
        nn.ConvTranspose2d(128, 1, kernel_size=(3, 3), padding=(1, 1), bias=False),
        nn.Tanh()
    ]
    layers = layer1 + layer2 + layer3
    model = nn.Sequential(*layers)
    return model
input_size = 100
hidden_size = 7*7*256 
output_size = 0
generator = generator_model(input_size, hidden_size, output_size)
print(generator)

Sequential(
  (0): Linear(in_features=100, out_features=12544, bias=False)
  (1): BatchNorm1d(12544, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01, inplace=True)
  (3): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.01, inplace=True)
  (6): ConvTranspose2d(128, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (7): Tanh()
)


In [13]:
def discriminator_model():
    layer1 = [
        nn.Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
        nn.LeakyReLU(inplace=True),
        nn.Dropout(0.3)
    ]
    layer2 = [
        nn.Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
        nn.LeakyReLU(inplace=True),
        nn.Dropout(0.3)
            
    ]
    layer3 = [
        nn.Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
        nn.LeakyReLU(inplace=True),
        nn.Dropout(0.3)
    ]
    layers = layer1 + layer2 + layer3 + [nn.Flatten(), nn.Linear(256 * 4 * 4, 1)]

    model = nn.Sequential(*layers)
    return model
discriminator = discriminator_model()
print(discriminator)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (1): LeakyReLU(negative_slope=0.01, inplace=True)
  (2): Dropout(p=0.3, inplace=False)
  (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (4): LeakyReLU(negative_slope=0.01, inplace=True)
  (5): Dropout(p=0.3, inplace=False)
  (6): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (7): LeakyReLU(negative_slope=0.01, inplace=True)
  (8): Dropout(p=0.3, inplace=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=4096, out_features=1, bias=True)
)
